Hola **Fernando**!

Soy **Patricio Requena** 👋. Es un placer ser el revisor de tu proyecto el día de hoy!

Revisaré tu proyecto detenidamente con el objetivo de ayudarte a mejorar y perfeccionar tus habilidades. Durante mi revisión, identificaré áreas donde puedas hacer mejoras en tu código, señalando específicamente qué y cómo podrías ajustar para optimizar el rendimiento y la claridad de tu proyecto. Además, es importante para mí destacar los aspectos que has manejado excepcionalmente bien. Reconocer tus fortalezas te ayudará a entender qué técnicas y métodos están funcionando a tu favor y cómo puedes aplicarlos en futuras tareas. 

_**Recuerda que al final de este notebook encontrarás un comentario general de mi parte**_, empecemos!

Encontrarás mis comentarios dentro de cajas verdes, amarillas o rojas, ⚠️ **por favor, no muevas, modifiques o borres mis comentarios** ⚠️:


<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
Si todo está perfecto.
</div>

<div class="alert alert-block alert-warning">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
Si tu código está bien pero se puede mejorar o hay algún detalle que le hace falta.
</div>

<div class="alert alert-block alert-danger">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
Si de pronto hace falta algo o existe algún problema con tu código o conclusiones.
</div>

Puedes responderme de esta forma:
<div class="alert alert-block alert-info">
<b>Respuesta del estudiante</b> <a class=“tocSkip”></a>


## Paso 1 - Importamos las librerias

In [39]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.utils import resample
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, roc_auc_score, classification_report, confusion_matrix

RND = 42

## Paso 2 - Cargamos datos y revisamos

In [40]:
df = pd.read_csv('/datasets/Churn.csv')
print(df.shape)
df.head()

(10000, 14)


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


## Paso 3 - Reemplazar infinitos, revisar nulos y balance de la clase objetivo

In [41]:
# Reemplazar infinitos por NaN (evita errores)
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Revisar nulos por columna
print(df.isna().sum())

# Ver distribución de la variable objetivo
print(df['Exited'].value_counts())
print(df['Exited'].value_counts(normalize=True).round(3))

RowNumber            0
CustomerId           0
Surname              0
CreditScore          0
Geography            0
Gender               0
Age                  0
Tenure             909
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
dtype: int64
0    7963
1    2037
Name: Exited, dtype: int64
0    0.796
1    0.204
Name: Exited, dtype: float64


### Conclusión

Detectamos si hay nulos y confirmamos desbalance (habitual en este dataset).

<div class="alert alert-block alert-success">
<b>Comentario del revisor (1ra Iteracion)</b> <a class=“tocSkip”></a>

Correcto! Siempre en proyectos de clasificación utilizando Machine Learning hay que revisar el balance de las clases ya que de esto dependerá las técnicas a aplicar o los modelos a utilizar para conseguir un buen desempeño en las predicciones
</div>

## Paso 4 — Quitar columnas irrelevantes y codificar categorías

In [42]:
# eliminar columnas irrelevantes
df = df.drop(columns=['RowNumber', 'CustomerId', 'Surname'])

# convertir categóricas a dummies (Geography, Gender)
df = pd.get_dummies(df, columns=['Geography', 'Gender'], drop_first=True)

# separar X e y
X = df.drop(columns=['Exited'])
y = df['Exited']

### Conclusión

X y y listos para partir en conjuntos.

## Paso 5 - Dividir estratificado: train / val / test (60/20/20)

In [43]:
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.20, stratify=y, random_state=RND)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, stratify=y_temp, random_state=RND)

print("Tamaños train/val/test:", X_train.shape[0], X_val.shape[0], X_test.shape[0])
print("Proporción en train:", y_train.value_counts(normalize=True).round(3))

Tamaños train/val/test: 6000 2000 2000
Proporción en train: 0    0.796
1    0.204
Name: Exited, dtype: float64


### Conclusión

Tenemos conjuntos independientes; test no se toca hasta el final.

## Paso 6 - Imputar NaN usando la media del entrenamiento

In [44]:
# SimpleImputer: aprender media en X_train y aplicar a val/test
imputer = SimpleImputer(strategy='mean')

# fit en X_train, transform en train/val/test
X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=X.columns, index=X_train.index)
X_val   = pd.DataFrame(imputer.transform(X_val),   columns=X.columns, index=X_val.index)
X_test  = pd.DataFrame(imputer.transform(X_test),  columns=X.columns, index=X_test.index)

# Confirmar que ya no hay NaN
print("NaN en train/val/test:", X_train.isna().sum().sum(), X_val.isna().sum().sum(), X_test.isna().sum().sum())

NaN en train/val/test: 0 0 0


### Conclusión

Ahora los datos no producirán el error de NaN al entrenar.

## Paso 7 - Baseline simple (sin corrección del desbalance)

In [45]:
# Baseline RandomForest (sin cambios)
rf = RandomForestClassifier(random_state=RND)
rf.fit(X_train, y_train)

# Baseline LogisticRegression con escalado en pipeline
pipe_lr = Pipeline([('scaler', StandardScaler()), ('lr', LogisticRegression(max_iter=2000, random_state=RND))])
pipe_lr.fit(X_train, y_train)

# Evaluar en validation
y_val_pred_rf = rf.predict(X_val)
y_val_proba_rf = rf.predict_proba(X_val)[:,1]

y_val_pred_lr = pipe_lr.predict(X_val)
y_val_proba_lr = pipe_lr.predict_proba(X_val)[:,1]

print("RF baseline - F1 (val):", f1_score(y_val, y_val_pred_rf), "AUC (val):", roc_auc_score(y_val, y_val_proba_rf))
print("LR baseline - F1 (val):", f1_score(y_val, y_val_pred_lr), "AUC (val):", roc_auc_score(y_val, y_val_proba_lr))

RF baseline - F1 (val): 0.561622464898596 AUC (val): 0.853280861755438
LR baseline - F1 (val): 0.3189964157706094 AUC (val): 0.7561907053432477


<div class="alert alert-block alert-success">
<b>Comentario del revisor (1ra Iteracion)</b> <a class=“tocSkip”></a>

Correccto, se puede ver que el desbalance afecta el desempeño de los modelos
</div>

### Conclusión

Anota F1 y AUC-ROC como baseline.

## Paso 8 - Método 1: class_weight='balanced'

In [46]:
# RandomForest con class_weight
rf_cw = RandomForestClassifier(class_weight='balanced', random_state=RND)
rf_cw.fit(X_train, y_train)

# LogisticRegression con class_weight en pipeline
pipe_lr_cw = Pipeline([('scaler', StandardScaler()),
                       ('lr', LogisticRegression(class_weight='balanced', max_iter=2000, random_state=RND))])
pipe_lr_cw.fit(X_train, y_train)

# Evaluar en validation
y_val_pred_rf_cw = rf_cw.predict(X_val)
y_val_proba_rf_cw = rf_cw.predict_proba(X_val)[:,1]

y_val_pred_lr_cw = pipe_lr_cw.predict(X_val)
y_val_proba_lr_cw = pipe_lr_cw.predict_proba(X_val)[:,1]

print("RF class_weight - F1 (val):", f1_score(y_val, y_val_pred_rf_cw), "AUC:", roc_auc_score(y_val, y_val_proba_rf_cw))
print("LR class_weight - F1 (val):", f1_score(y_val, y_val_pred_lr_cw), "AUC:", roc_auc_score(y_val, y_val_proba_lr_cw))

RF class_weight - F1 (val): 0.5632 AUC: 0.856003923800534
LR class_weight - F1 (val): 0.48 AUC: 0.7606898115372692


### Conclusión

Compara estos F1 con el baseline para ver si mejora.

## Paso 9 - Método 2A: Undersampling (reducir clase mayoritaria) con resample

In [47]:
# Preparar DataFrame de entrenamiento
y_train.name = 'Exited'
train_df = pd.concat([X_train.reset_index(drop=True), y_train.reset_index(drop=True)], axis=1)

# separar clases
no_churn = train_df[train_df['Exited'] == 0]
churn    = train_df[train_df['Exited'] == 1]

# undersample: reducir no_churn al tamaño de churn
no_churn_down = resample(no_churn,
                         replace=False,
                         n_samples=len(churn),
                         random_state=RND)

train_under = pd.concat([no_churn_down, churn]).sample(frac=1, random_state=RND)  # barajar
X_train_under = train_under.drop(columns=['Exited'])
y_train_under = train_under['Exited']

# entrenar un RF sobre los datos undersampleados
rf_under = RandomForestClassifier(random_state=RND)
rf_under.fit(X_train_under, y_train_under)

# evaluar
y_val_pred_under = rf_under.predict(X_val)
y_val_proba_under = rf_under.predict_proba(X_val)[:,1]
print("RF undersample - F1 (val):", f1_score(y_val, y_val_pred_under), "AUC:", roc_auc_score(y_val, y_val_proba_under))

RF undersample - F1 (val): 0.586894586894587 AUC: 0.8514330971958091


### Conclusión

Puede subir recall, pero a veces reduce la información (porque borramos datos).

<div class="alert alert-block alert-success">
<b>Comentario del revisor (1ra Iteracion)</b> <a class=“tocSkip”></a>

Perfecto, técnicas cómo sobre-muestreo o sub-muestreo o la modificación de parámetros en algunos modelos para especificar el desbalance ayuda a los modelos a poder generalizar mejor cada caso y obtener un buen desempeño
</div>

## Paso 10 - Método 2B: Oversampling simple (repetir casos minoritarios) con resample

In [48]:
# oversample: repetir churn hasta igualar no_churn
churn_up = resample(churn,
                    replace=True,
                    n_samples=len(no_churn),
                    random_state=RND)

train_over = pd.concat([no_churn, churn_up]).sample(frac=1, random_state=RND)
X_train_over = train_over.drop(columns=['Exited'])
y_train_over = train_over['Exited']

# entrenar RF sobre oversampled
rf_over = RandomForestClassifier(random_state=RND)
rf_over.fit(X_train_over, y_train_over)

# evaluar
y_val_pred_over = rf_over.predict(X_val)
y_val_proba_over = rf_over.predict_proba(X_val)[:,1]
print("RF oversample - F1 (val):", f1_score(y_val, y_val_pred_over), "AUC:", roc_auc_score(y_val, y_val_proba_over))

RF oversample - F1 (val): 0.6179775280898876 AUC: 0.8551548466802704


### Conclusión

Sencillo y fácil de explicar; puede ayudar si la clase minoritaria es pequeña.

## Paso 11 - Comparar resultados (val) y elegir el mejor según F1

In [49]:
print("Baseline RF F1:", f1_score(y_val, y_val_pred_rf))
print("RF class_weight F1:", f1_score(y_val, y_val_pred_rf_cw))
print("RF undersample F1:", f1_score(y_val, y_val_pred_under))
print("RF oversample F1:", f1_score(y_val, y_val_pred_over))

Baseline RF F1: 0.561622464898596
RF class_weight F1: 0.5632
RF undersample F1: 0.586894586894587
RF oversample F1: 0.6179775280898876


### Conclusión

Elige best_model (por ejemplo rf_over o rf_cw) que tenga mayor F1 en validación.

## Paso 12 - Afinar umbral (threshold) del mejor modelo para maximizar F1

In [50]:
# Supongamos que 'best_model' es rf_over; ajusta si elegiste otro
best_model = rf_over   # cambia si tu mejor fue otro
probs_val = best_model.predict_proba(X_val)[:,1]

best_f1 = 0
best_thresh = 0.5
for t in np.linspace(0.01, 0.99, 99):
    preds = (probs_val >= t).astype(int)
    f = f1_score(y_val, preds)
    if f > best_f1:
        best_f1 = f
        best_thresh = t

print("Mejor umbral en val:", best_thresh, "F1:", best_f1)

Mejor umbral en val: 0.46 F1: 0.6230366492146596


### Conclusión

Usaremos best_thresh para evaluar en test.

## Paso 13 - Evaluación final en TEST (usar umbral optimizado)

In [51]:
probs_test = best_model.predict_proba(X_test)[:,1]
y_test_pred = (probs_test >= best_thresh).astype(int)

print("F1 (test):", f1_score(y_test, y_test_pred))
print("AUC-ROC (test):", roc_auc_score(y_test, probs_test))
print(classification_report(y_test, y_test_pred, digits=4))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_test_pred))

F1 (test): 0.6047120418848168
AUC-ROC (test): 0.8459160238821256
              precision    recall  f1-score   support

           0     0.8929    0.9209    0.9067      1593
           1     0.6471    0.5676    0.6047       407

    accuracy                         0.8490      2000
   macro avg     0.7700    0.7442    0.7557      2000
weighted avg     0.8429    0.8490    0.8452      2000

Matriz de confusión:
 [[1467  126]
 [ 176  231]]


### Conclusión

Estas son tus métricas finales que presentarás (F1 y AUC-ROC). Si F1 ≥ 0.59, cumpliste la condición del proyecto.

## Paso 14 - Conclusiones generales

- **Preprocesado:** eliminé identificadores, apliqué get_dummies, reemplacé inf por NaN y los imputé con la media calculada sobre el conjunto de entrenamiento.

- **Investigación del desbalance:** Exited está desbalanceado (mostrar value_counts()), por lo que probé varios métodos.

- **Baselines:** entrené RandomForest y LogisticRegression sin corrección; obtuve F1 y AUC-ROC iniciales.

- **Corrección de desbalance (mínimo 2):**
1. **class_weight='balanced'** (sin alterar datos).

2. **resample de sklearn:** undersampling (reducir clase mayoritaria) y oversampling simple (duplicar clase minoritaria).

- **Selección y ajuste:** elegí el modelo con mayor F1 en validación y optimicé el umbral de probabilidad para maximizar F1.

- **Resultados finales:** presentar F1 (test) y AUC-ROC (test) y la matriz de confusión.

<div class="alert alert-block alert-success">
<b>Comentario del revisor (1ra Iteracion)</b> <a class=“tocSkip”></a>

Te felicito por el trabajo realizado Fernando, se nota que conoces las métricas de evaluación y planteas conclusiones muy acertadas en cuanto a los resultados. Es importante siempre en un problema de ML identificar el balance entre las clases de tu variable objetivo para poder escoger la métrica correcta ya que para datasets desbalanceados la métrica de accuracy suele presentar buenos valores pero en realidad las predicciones no son buenas y es por la forma en cómo se calcula esta métrica que puede llevar a interpretaciones erroneas en cambio el f1-score es la ideal para este tipo de casos ya que maneja mejor este tipo de problemas.
    
Saludos!
</div>